In [ ]:
import numpy as np
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import contractions
import unidecode
import re
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import string
from sklearn.utils import shuffle
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification,RobertaTokenizer
from transformers import EarlyStoppingCallback
# from gensim.parsing.preprocessing import remove_stopwords
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import os
# from cleantext import clean

/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentence

# def removeUser(sentence):
#     remove = re.compile('(\s*)user(\s*)')
#     sentence = remove.sub(" ", sentence)
#     return sentence

# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1


In [ ]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
    # data[sent_or_word] = data[sent_or_word].apply(lambda x: removeEmoji(x))
    # data[sent_or_word] = data[sent_or_word].apply(lambda x: removeUser(x))


    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels'])

    return x,y


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}


In [ ]:
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}

max_length = 128
num_labels = 3
model_name = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes = True)

In [ ]:
allTrain = os.listdir('emojiRemovedFiles/train')
for file in allTrain:
    newModelNameForSaving = model_name.split("/")[-1]

    train_data = pd.read_csv('emojiRemovedFiles/train/{}'.format(file), sep = '\t')
    val_data = pd.read_csv('emojiRemovedFiles/val/{}'.format(file) ,sep = '\t')
    train_data = train_data[train_data['translation_sentence_level'].notna()]
    val_data = val_data[val_data['translation_sentence_level'].notna()]
    train_data['categorical_labels'] = train_data['label'].map(add_labels)
    val_data['categorical_labels'] = val_data['label'].map(add_labels)


    x_train, y_train = preprocessing(train_data)
    x_val, y_val = preprocessing(val_data)

    x_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
    x_val_tokenized = tokenizer(x_val, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)

    train_dataset = Dataset(x_train_tokenized, y_train)
    val_dataset = Dataset(x_val_tokenized, y_val)


    def compute_metrics(p):
        pred, labels = p
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
        precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
        f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    args = TrainingArguments(
        output_dir = "checkpointsForRemoveEmojiTwitterRoberta/{}_{}".format(newModelNameForSaving, file.split(".")[0]),
        evaluation_strategy="epoch",
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        num_train_epochs=5,
        seed=7,
        load_best_model_at_end=True,
        save_strategy='epoch',
        overwrite_output_dir=True)
        #save_strategy = "no"

    trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset = val_dataset,
      compute_metrics = compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])


    trainer.train()
    trainer.save_model("modelForRemoveEmojiTwitterRoberta/{}_{}".format(newModelNameForSaving, file.split(".")[0]))

    test_data = pd.read_csv("emojiRemovedFiles/test/{}".format(file),sep='\t')
    test_data = test_data[test_data['translation_sentence_level'].notna()]

    test_data['categorical_labels'] = test_data['label'].map(add_labels)
    print(test_data['translation_sentence_level'].isnull().sum())
    X_test, y_test = preprocessing(test_data)


    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)


    test_dataset = Dataset(X_test_tokenized, y_test)


    predictions = trainer.predict(test_dataset)

    pred = np.argmax(predictions.predictions, axis=-1)


    accuracy = accuracy_score(y_true=y_test, y_pred=pred)
    recall = recall_score(y_true=y_test, y_pred=pred,average='weighted')
    precision = precision_score(y_true=y_test, y_pred=pred,average='weighted')
    f1 = f1_score(y_true=y_test, y_pred=pred,average='weighted')


    model_results_dict['Model_Name'].append(file)
    model_results_dict['Accuracy'].append(accuracy)
    model_results_dict['Recall'].append(recall)
    model_results_dict['Precision'].append(precision)
    model_results_dict['f1'].append(f1)

    df = pd.DataFrame(data = model_results_dict)
    df.to_csv("{}_TwitterRobertaRemoveEmojiresults.csv".format(newModelNameForSaving))


model_results_dict

In [ ]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))

    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels'])
    return x,y

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
# ------------------------------------------------------------------------------------------------------------------------------------------------
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}

max_length = 128
num_labels = 3
model_name = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes = True)
# ------------------------------------------------------------------------------------------------------------------------------------------------

allTrain = os.listdir('emojiRemovedFiles/train')
for file in allTrain:
    newModelNameForSaving = model_name.split("/")[-1]

    train_data = pd.read_csv('emojiRemovedFiles/train/{}'.format(file), sep = '\t')
    val_data = pd.read_csv('emojiRemovedFiles/val/{}'.format(file) ,sep = '\t')

    train_data = train_data[train_data['translation_sentence_level'].notna()]
    val_data = val_data[val_data['translation_sentence_level'].notna()]

    train_data['categorical_labels'] = train_data['label'].map(add_labels)
    val_data['categorical_labels'] = val_data['label'].map(add_labels)


    x_train, y_train = preprocessing(train_data)
    x_val, y_val = preprocessing(val_data)

    x_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
    x_val_tokenized = tokenizer(x_val, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)

    train_dataset = Dataset(x_train_tokenized, y_train)
    val_dataset = Dataset(x_val_tokenized, y_val)


    def compute_metrics(p):
        pred, labels = p
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
        precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
        f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    args = TrainingArguments(
        output_dir = "checkpointsForRemoveEmojiTwitterRoberta/{}_{}".format(newModelNameForSaving, file.split(".")[0]),
        evaluation_strategy="epoch",
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        num_train_epochs=5,
        seed=7,
        load_best_model_at_end=True,
        save_strategy='epoch',
        overwrite_output_dir=True)
        #save_strategy = "no"

    trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset = val_dataset,
      compute_metrics = compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])


    trainer.train()
    trainer.save_model("modelForRemoveEmojiTwitterRoberta/{}_{}".format(newModelNameForSaving, file.split(".")[0]))

    test_data = pd.read_csv("emojiRemovedFiles/test/{}".format(file),sep='\t')
    test_data = test_data[test_data['translation_sentence_level'].notna()]

    test_data['categorical_labels'] = test_data['label'].map(add_labels)
    print(test_data['translation_sentence_level'].isnull().sum())
    X_test, y_test = preprocessing(test_data)

    X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
    test_dataset = Dataset(X_test_tokenized, y_test)
    predictions = trainer.predict(test_dataset)
    pred = np.argmax(predictions.predictions, axis=-1)

    accuracy = accuracy_score(y_true=y_test, y_pred=pred)
    recall = recall_score(y_true=y_test, y_pred=pred,average='weighted')
    precision = precision_score(y_true=y_test, y_pred=pred,average='weighted')
    f1 = f1_score(y_true=y_test, y_pred=pred,average='weighted')

    model_results_dict['Model_Name'].append(file)
    model_results_dict['Accuracy'].append(accuracy)
    model_results_dict['Recall'].append(recall)
    model_results_dict['Precision'].append(precision)
    model_results_dict['f1'].append(f1)

    df = pd.DataFrame(data = model_results_dict)
    df.to_csv("{}_TwitterRobertaRemoveEmojiresults.csv".format(newModelNameForSaving))